# 指令發送 GUI

包含安裝 ipywidgets 的指令。執行一次後即可跳過此步驟。首次安裝後需要重新啟動 kernel 並重新載入 JupyterLab 的瀏覽器頁面。

In [ ]:
import asyncio

import ipywidgets as widgets
import kachaka_api

client = kachaka_api.aio.KachakaApiClient()

# Build UI

NAMES = [
    "MoveShelf",
    "ReturnShelf",
    "UndockShelf",
    "MoveToLocation",
    "ReturnHome",
    "DockShelf",
    "Speak",
    "MoveToPose",
]
command_type_radio = widgets.RadioButtons(
    options=NAMES,
    layout={"width": "max-content"},
    description="Command Type",
)
cancel_all_checkbox = widgets.Checkbox(description="Cancel All")
tts_on_success_text = widgets.Text(placeholder="成功時播報的語句")
title_text = widgets.Text(placeholder="標題")
shelf_picker = widgets.Dropdown(description="Shelf:")
location_picker = widgets.Dropdown(description="Location:")
speak_text = widgets.Text(placeholder="播報的語句")
move_to_pose_x = widgets.FloatText(description="X:", step=0.1)
move_to_pose_y = widgets.FloatText(description="Y:", step=0.1)
move_to_pose_yaw = widgets.FloatText(description="Yaw:", step=0.1)
no_option_label = widgets.Label("無選項")
options_stack = widgets.Stack(
    [
        widgets.VBox([widgets.Label("MoveShelf"), shelf_picker, location_picker]),
        widgets.VBox([widgets.Label("ReturnShelf"), shelf_picker]),
        widgets.VBox([widgets.Label("UndockShelf"), no_option_label]),
        widgets.VBox([widgets.Label("MoveToLocation"), location_picker]),
        widgets.VBox([widgets.Label("ReturnHome"), no_option_label]),
        widgets.VBox([widgets.Label("DockShelf"), no_option_label]),
        widgets.VBox([widgets.Label("Speak"), speak_text]),
        widgets.VBox(
            [
                widgets.Label("MoveToPose"),
                move_to_pose_x,
                move_to_pose_y,
                move_to_pose_yaw,
            ]
        ),
    ],
    selected_index=0,
)
send_button = widgets.Button(description="Send")
ui = widgets.HBox(
    [
        widgets.VBox(
            [
                title_text,
                tts_on_success_text,
                cancel_all_checkbox,
                send_button,
            ]
        ),
        command_type_radio,
        options_stack,
    ]
)

# Setup event handlers


def on_radio_change(evt):
    options_stack.selected_index = command_type_radio.options.index(evt.new)


command_type_radio.observe(on_radio_change, names="value")


async def on_send():
    cancel_all = cancel_all_checkbox.value
    tts_on_success = tts_on_success_text.value
    title = title_text.value
    if command_type_radio.value == "MoveShelf":
        await client.move_shelf(
            shelf_picker.value,
            location_picker.value,
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )
    elif command_type_radio.value == "ReturnShelf":
        await client.return_shelf(
            shelf_picker.value,
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )
    elif command_type_radio.value == "UndockShelf":
        await client.undock_shelf(
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )
    elif command_type_radio.value == "MoveToLocation":
        await client.move_to_location(
            location_picker.value,
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )
    elif command_type_radio.value == "ReturnHome":
        await client.return_home(
            cancel_all=cancel_all, tts_on_success=tts_on_success, title=title
        )
    elif command_type_radio.value == "DockShelf":
        await client.dock_shelf(
            cancel_all=cancel_all, tts_on_success=tts_on_success, title=title
        )
    elif command_type_radio.value == "Speak":
        await client.speak(
            speak_text.value,
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )
    elif command_type_radio.value == "MoveToPose":
        await client.move_to_pose(
            move_to_pose_x.value,
            move_to_pose_y.value,
            move_to_pose_yaw.value,
            cancel_all=cancel_all,
            tts_on_success=tts_on_success,
            title=title,
        )


async def on_shelves_update(shelves) -> None:
    shelf_picker.options = {f"{s.name} ({s.id})": s.id for s in shelves}


async def on_locations_update(locations) -> None:
    location_picker.options = {f"{l.name} ({l.id})": l.id for l in locations}


client.set_shelves_callback(on_shelves_update)
client.set_locations_callback(on_locations_update)

send_button.on_click(lambda _: asyncio.create_task(on_send()))

# Show UI

ui